# Test/Debuger for the predict.py file

In [51]:
import os
import torch
import segmentation_models_pytorch as smp
import yaml
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import json
from PIL import Image, ImageOps

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Define the transformation to apply to the image
transform = Compose([
    ToTensor(),  # Convert the image to a tensor
])

In [66]:
# Step 1: Load the config file
config_path = '../runs/unet_smp/demo_run/config/config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

In [67]:
# Extract values from the config
checkpoint_path = os.path.join(config['path_checkpoints'], 'checkpoint_epoch100.pth')
data_root = config['path_data']
image_size = config['image_size']
in_channels = config['in_channels']
out_channels = config['out_channels']
encoder_name = config['encoder_name']
encoder_weights = config['encoder_weights']
image_folder = os.path.join(config['path_data'], 'images')

In [68]:
# Step 2: Load the pre-trained model with checkpoint
model = smp.Unet(
    encoder_name=encoder_name,                   # Encoder architecture, e.g., resnet34
    encoder_weights=encoder_weights,             # Pre-trained weights from ImageNet
    in_channels=in_channels,                     # Number of input channels (3 for RGB)
    out_channels=out_channels,                   # Number of output channels (1 for binary)
    activation='sigmoid'                         # Sigmoid activation for binary segmentation
)

In [70]:
# Step 4: Create a custom dataset for validation
annotations_path = os.path.join(data_root, 'annotations/val.json')

with open(annotations_path) as f:
    annotations = json.load(f)

# Extract relevant parts of the annotations
image_annotations = {annotation['image_id']: annotation for annotation in annotations['annotations']}
image_info = {image['id']: image for image in annotations['images']}

In [72]:
# Load the model weights from checkpoint
model.load_state_dict(torch.load(checkpoint_path))
model.eval()  # Set the model to evaluation mode

/tmp/ipykernel_127215/194798669.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [81]:
# Padding function
def pad_to_divisible(image, divisor=32):
    width, height = image.size
    pad_width = (divisor - width % divisor) % divisor
    pad_height = (divisor - height % divisor) % divisor
    padding = (0, 0, pad_width, pad_height)  # Right and bottom padding
    padded_image = Image.new('RGB', (width + pad_width, height + pad_height))
    padded_image.paste(image, (0, 0))  # Paste original image on top-left corner
    return padded_image

# Function to get the annotation for an image by its ID
def get_annotation_by_image_id(image_id):
    return image_annotations.get(image_id, None)

# Function to predict and visualize the result
def predict_and_visualize(image_path, annotation=None):
    # Load the image
    image = Image.open(image_path).convert("RGB")
    
    # Pad image to ensure dimensions are divisible by 32
    image = pad_to_divisible(image, divisor=32)

    # Apply transformations
    image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device
    
    # Perform prediction
    with torch.no_grad():
        output = model(image_tensor)  # Forward pass
    
    # Sigmoid for binary segmentation (if required)
    output_mask = torch.sigmoid(output).squeeze().cpu().numpy()  # Move output back to CPU for visualization

    # Visualize the result
    plt.figure(figsize=(12, 6))

    # Display original image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")

    # Display predicted mask
    plt.subplot(1, 2, 2)
    plt.imshow(output_mask, cmap='gray')
    plt.title("Predicted Mask")

    # If you have a segmentation mask in the annotation, visualize it too
    if annotation:
        ground_truth_mask = np.zeros_like(output_mask)
        for seg in annotation.get('segmentation', []):  # 'segmentation' could be polygon data
            if isinstance(seg, list):
                polygon = np.array(seg).reshape((-1, 2))  # Assuming polygon format is [x1, y1, x2, y2, ...]
                plt.fill(polygon[:, 0], polygon[:, 1], color='red', alpha=0.3)  # Example for polygon visualization

        # Display ground truth mask (optional)
        plt.subplot(1, 2, 2)
        plt.imshow(ground_truth_mask, cmap='gray', alpha=0.5)
        plt.title("Ground Truth Mask")

    plt.show()

In [82]:
# Example usage
image_id = 31  # pick an image
annotation = get_annotation_by_image_id(image_id)  # Get the annotation for that image
predict_and_visualize(image_id, annotation)


AttributeError: 'int' object has no attribute 'read'